In [1]:
import numpy as np

In [9]:
def gini(y):
    classes,counts = np.unique(y,return_counts=True)
    p = counts/counts.sum()
    return (1 - np.sum(p**2))

def best_split(X,y):
    n_samples , n_features = X.shape
    best_feature , best_threshold , best_gain = None , None , -1

    parent_impurity = gini(y)

    for feature in range(n_features):
        threshold = np.unique(X[:,feature])
        for t in threshold:
            left_idx = X[:, feature]<= t
            right_idx = ~left_idx

            if left_idx.sum() == 0 or right_idx.sum() == 0:
                continue
            n = len(y)

            left_impurity = gini(y[left_idx])
            right_impurity = gini(y[right_idx])

            weighted_impurity = ((len(y[left_idx] / n)) * left_impurity + (len(y[right_idx] / n)) * right_impurity )

            gain = parent_impurity - weighted_impurity

            if gain > best_gain :
                best_feature = feature
                best_threshold = t
                best_gain = gain


    return best_feature , best_threshold, best_gain


# Node Class 
class Node :
    def __init__(self, feature = None , threshold = None , left = None , right = None , value = None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf(self):
        return self.value is not None

class DecisionTree :
    def __init__(self , max_depth = 5):
        self.max_depth = max_depth

    def fit(self , X ,y):
        self.root = self._grow_tree(X,y)

    def _grow_tree(self,X,y , depth=0):
        if depth >= self.max_depth or len(np.unique(y)) == 1 :
            node_val = self._majority_class(y)
            node = Node(value = node_val)
            return node

        feature,threshold , gain = best_split(X,y)
        
        if gain == -1:
            return Node(value = self._majority_class(y))

        left_idx = X[:, feature] <= threshold
        right_idx = ~left_idx
        left_tree = self._grow_tree(X[left_idx] , y[left_idx] , depth+1)
        right_tree = self._grow_tree(X[right_idx] , y[right_idx] , depth+1)
        return Node(feature=feature, threshold = threshold , left = left_tree, right = right_tree)

    def _majority_class(self,y):
        classes , counts = np.unique(y, return_counts = True)
        return classes[np.argmax(counts)]

    def predict_one(self,x , node=None):
        if node is None:
            node = self.root
        if node.is_leaf():
            return node.value
        if x[node.feature] <= node.threshold:
            return self.predict_one(x, node.left)
        else:
            return self.predict_one(x, node.right)

    def predict(self, X):
        return np.array([self.predict_one(x) for x in X])
                                                                         
            

In [10]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Use Iris dataset
iris = load_iris()
X, y = iris.data, iris.target

# Binary classification (setosa vs not)
y = (y == 0).astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

tree = DecisionTree(max_depth=3)
tree.fit(X_train, y_train)

y_pred = tree.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 1.0
